In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

2023-11-07 16:07:21.582930: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 16:07:21.612244: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 16:07:21.612275: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 16:07:21.612305: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 16:07:21.618222: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 16:07:21.618958: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
y_train = np.where(y_train != 1, 0, y_train)
y_test = np.where(y_test != 1, 0, y_test)

In [ ]:
'''from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}'''

In [4]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
model = ResNet50(weights='imagenet', include_top=False)

In [5]:
for layer in model.layers:
    layer.trainable = False
averaging_layer = layers.GlobalAveragePooling2D()(model.output)
output_layer = layers.Dense(1, activation='sigmoid')(averaging_layer)


In [6]:
model = models.Model(inputs=model.input, outputs=output_layer)

In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
'''from skimage.transform import resize

new_size = (224, 224)

X_train_resized = np.empty((X_train.shape[0], new_size[0], new_size[1], 3))
X_test_resized = np.empty((X_test.shape[0], new_size[0], new_size[1], 3))

for i in range(X_train.shape[0]):
    X_train_resized[i] = resize(X_train[i], new_size, preserve_range=True, mode='reflect')
for i in range(X_test.shape[0]):
    X_test_resized[i] = resize(X_test[i], new_size, preserve_range=True, mode='reflect')
'''

In [ ]:
'''with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "wb") as file:
    pkl.dump(X_train_resized, file=file)
file.close()
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "wb") as file:
    pkl.dump(X_test_resized, file=file)
file.close()'''

In [8]:
import pickle as pkl
with open("/home/opc/block_vol/cifar_10_resized_224/X_train.pkl", "rb") as file:
    X_train_resized=pkl.load(file=file)
file.close()
with open("/home/opc/block_vol/cifar_10_resized_224/X_test.pkl", "rb") as file:
    X_test_resized=pkl.load(file=file)
file.close()

In [9]:
model.fit(X_train_resized, y_train)

2023-11-07 16:10:01.426218: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 30105600000 exceeds 10% of free system memory.


1563/1563 [==============================] - 542s 345ms/step - loss: 0.0735 - accuracy: 0.9729


In [10]:
y_pred = model.predict(X_test_resized)

313/313 [==============================] - 108s 344ms/step


In [16]:
y_pred

array([[9.0717250e-01],
       [4.1750107e-02],
       [4.9628514e-01],
       ...,
       [4.9717717e-05],
       [8.9232886e-01],
       [2.5247880e-03]], dtype=float32)

In [20]:
#y_pred_argmax = [np.argmax(y) for y in y_pred]
from sklearn.metrics import accuracy_score
predictions = (y_pred >= 0.5).astype(int) # return the truth value of that statement. brilliant.
accuracy_score(y_true=y_test, y_pred=predictions)

0.9798